In [1]:
import os, shutil
train_dir = '../train'
validation_dir = '../validation'
test_dir = '../test'

In [2]:
from keras.utils import image_dataset_from_directory

IMG_SIZE = 32

train_dataset = image_dataset_from_directory(train_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE),batch_size=512)
validation_dataset = image_dataset_from_directory(validation_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE),batch_size=128)
test_dataset = image_dataset_from_directory(test_dir,label_mode='categorical',image_size=(IMG_SIZE, IMG_SIZE),batch_size=128)

2024-05-27 16:58:24.150707: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 40000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.


In [3]:
from keras.applications.vgg16 import VGG16

conv_base = VGG16(weights="imagenet", include_top=False) 
conv_base.trainable = False

In [4]:
from tensorflow import keras
from keras import layers

data_augmentation = keras.Sequential(
[
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
]
)

inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [6]:
model.compile(loss="categorical_crossentropy",optimizer="rmsprop", metrics=["accuracy"])

history = model.fit(train_dataset,epochs=50,validation_data=validation_dataset)

Epoch 1/50
79/79 [==============================] - 9s 99ms/step - loss: 100.2420 - accuracy: 0.1552 - val_loss: 55.8069 - val_accuracy: 0.1528
Epoch 2/50
79/79 [==============================] - 8s 98ms/step - loss: 68.4978 - accuracy: 0.1540 - val_loss: 116.0304 - val_accuracy: 0.1094
Epoch 3/50
79/79 [==============================] - 6s 72ms/step - loss: 71.7579 - accuracy: 0.1535 - val_loss: 62.1108 - val_accuracy: 0.1541
Epoch 4/50
79/79 [==============================] - 6s 69ms/step - loss: 68.6065 - accuracy: 0.1585 - val_loss: 76.2373 - val_accuracy: 0.1337
Epoch 5/50
79/79 [==============================] - 6s 73ms/step - loss: 70.4313 - accuracy: 0.1576 - val_loss: 77.0784 - val_accuracy: 0.1654
Epoch 6/50
79/79 [==============================] - 5s 63ms/step - loss: 71.5386 - accuracy: 0.1528 - val_loss: 81.7996 - val_accuracy: 0.1177
Epoch 7/50
79/79 [==============================] - 6s 67ms/step - loss: 67.6849 - accuracy: 0.1580 - val_loss: 41.7150 - val_accuracy: 0.17

KeyboardInterrupt: 